# Analysing Admissions Essays: Unsupervised Approaches using scikit-learn

This notbook is designed to analyze every admissions essay submitted to Berkeley in the 2014-2015 academic year using Topic Modeling in Pythons scikit-learn package.  It begins with two CSV files (UTF-8) each containing unique and correspondeing dummie ID's for each of the two essay prompts that year.  There are column headers.

### Personal Statement 1
1. Freshmen: Describe the world you come from — for example, your family, community or school — and tell us how your world has shaped your dreams and aspirations.
1. Transfers: What is your intended major? Discuss how your interest in the subject developed and describe any experience you have had in the field — such as volunteer work, internships and employment, participation in student organizations and activities — and what you have gained from your involvement.

### Personal Statement 2
1. Tell us about a personal quality, talent, accomplishment, contribution or experience that is important to you. What about this quality or accomplishment makes you proud, and how does it relate to the person you are?

### Outline
1. [Setup the Analysis](#1.-Setup-the-Analysis)
  1. [Import Packages](#Import-Packages)
  1. [Important Questions](#Important-Questions)
  1. [Initialize Variables](#Initilize-Variables)
1. [Import and view the data using Pandas](#2.-Import-and-view-the-data-using-Pandas)
  1. [Import the data into a Pandas Dataframe](#Import-the-data-into-a-Pandas-Dataframe)
  1. [Lable the Columns](#Lable-the-Columns)
  1. [Merge the dataframes](#Merge-the-Dataframes)
  1. [Review the Data](#Review-the-data)
1. [Explore the Data & Drop missing values](#3.-Explore-the-Data-using-Pandas)
  1. [Are the ID's Unique?](#Are-the-ID's-Unique?)
  2. [Find Missing Data](#Find-missing-essays)
  1. [Drop the Missing Essays](#Drop-the-missing-essays)
1. [Pre-Processing the Essays](#4.-Pre-Processing-the-Essays)
  1. [Cleaning the text and tokenizing](#Cleaning-the-text-and-tokenizing)
  1. [Remove Stopwords](#Remove-Stopwords)
  1. [Stem the Tokens](#Stem-the-Tokens)
  1. [Testing Token Counts](#Testing-token-counts)
  1. [Convert Tokens back to Strings](#Convert-tokens-back-to-strings)
1. [Creating a sample for testing](#5.-Creating-a-sample-for-testing)
1. [Creating the DTM: scikit-learn](#6.-Creating-the-DTM:-scikit-learn)
1. [Tf-idf scores](#7.-Tf-idf-scores)
1. [Uncovering patterns using LDA](#8.-Uncovering-Patterns:-LDA)
  1. [Running the LDA Model](#Running-the-LDA-Model)
  1. [Format Topic Documnet Matrix](#Format-Topic-Document-Matrix)
  1. [Save the Documnet Topic Matrix](#Save-the-Documnet-Topic-Matrix)
  1. [Format Feature Matrix](#Format-Feature-Matrix)
  1. [Save the Feature Matrix](#Save-the-Feature-Matrix)
  1. [Reviewing Final Model Output](#Reviewing-Final-Model-Output)
1. [The End!](#9.-The-End!)
1. [Scratch](#10.-Scratch-Cell)

## 1. Setup the Analysis

Before we get started we must import a number of packages we'll need, answer a few important questions, and set up a few variables we'll need later.

### Import Packages

In [44]:
import pandas
import numpy
import time
import datetime
import platform
import ast # This is used below to asign the variables on the fly.
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import LatentDirichletAllocation
#import socket # I think this was to get the computer name, but may be changed now that platform is being used. see socket.gethostname()
#from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
print('All packages successfully imported!')

All packages successfully imported!


### Important Questions

In [45]:
#QUESTIONS TO ANSWER 
sample_yes_no = 'No'
number_of_topics = 4 # to revert to original, remove 'number_of_topics' from 'n_topics'.

### Initilize Variables

This initializes a few variables we'll need, and deals with a few of the answers to the questions we have here.

In [46]:
sample_yes_no = sample_yes_no.upper()
print('Analyse only a sample? '+sample_yes_no)

# These variables used later on in the script to write files
now = datetime.datetime.now()
time_for_f_name = now.strftime("%Y-%m-%d_at_%H-%M")

#This establishes the start time to calculate the time it takes to run the script
start_time = time.time()
print(start_time)

#Using platform package to access the machine name.
run_on = platform.node()

print(run_on)

Analyse only a sample? NO
1493580766.607309
BensMBP.local


## 2. Import and view the data using Pandas

First, we read our corpus, which is stored as a .csv file on our hard drive, into a Pandas dataframe. 

Note: Pandas is great for data munging and basic calculations because it's so easy to use, and its data structure is really intuitive. It's not memory efficient however, so you might quickly need to move away from it. 

### Import the data into a Pandas Dataframe

This next section automatically detects where the script is running and adjusts file paths accordingly.  

In [47]:
if (run_on == 'BensMBP') or (run_on == 'BensMBP.local'):
    df1 = pandas.read_csv("/Volumes/Extra Space/Google Drive/Scholarship/Writing Projects - Personal/Admissions Essays/Data/PS1_F16.csv", sep = ',', encoding = 'utf_8')
    df2 = pandas.read_csv("/Volumes/Extra Space/Google Drive/Scholarship/Writing Projects - Personal/Admissions Essays/Data/PS2_F16.csv", sep = ',', encoding = 'utf_8')
    print('The script is running locally.')
elif run_on == 'mercury':
    df1 = pandas.read_csv("../data/originals/PS1_F16.csv", sep = ',', encoding = 'utf_8')
    df2 = pandas.read_csv("../data/originals/PS2_F16.csv", sep = ',', encoding = 'utf_8')
    print('The script is running on mercury.')
else:
    print('The file path is unclear on this machine.')

The script is running locally.


In [48]:
#create dataframes called "df1" and "df2

#df1 = pandas.read_csv("/Volumes/Extra Space/Google Drive/Scholarship/Writing Projects - Personal/Admissions Essays/Data/PS1_F16.csv", sep = ',', encoding = 'utf_8')
#df2 = pandas.read_csv("/Volumes/Extra Space/Google Drive/Scholarship/Writing Projects - Personal/Admissions Essays/Data/PS2_F16.csv", sep = ',', encoding = 'utf_8')

#for the server
#df1 = pandas.read_csv("../data/originals/PS1_F16.csv", sep = ',', encoding = 'utf_8')
#df2 = pandas.read_csv("../data/originals/PS2_F16.csv", sep = ',', encoding = 'utf_8')


# View the dataframe.
# Notice the metadata. The column "Personal Statement 1 (RETIRED)" contains our text of interest.
# You can move the hashtag to view the other dataframe.
df1
# df2

,"﻿""ApplyUC Application CPID""",College,Personal Statement 1 (RETIRED)
0,3128896,College of Letters and Science,Bojio!\\That was what I playfully typed on my ...
1,3092833,College of Engineering,"My world is shaped by a 5' 10"", lean man, who ..."
2,3142974,College of Engineering,I come from a mediocre family in Malaysia. As ...
3,3020517,College of Letters and Science,Being a Mexican-American now is one of the mos...
4,3121294,College of Natural Resources,I am really confused when I have to fill out i...
5,3162551,College of Letters and Science,"Since before I was born, my parents wanted me ..."
6,3090215,College of Letters and Science,"When I think of my cousin Sid, I see him as th..."
7,3158702,College of Engineering,"I was born in a suburb just east of Seattle, t..."
8,3095465,College of Natural Resources,"Being the youngest in my family, I have been h..."
9,3147750,College of Letters and Science,Living up to the grand expectations of a cardi...


### Lable the Columns

Next we can rename the colum headers so they are easier to work with.

In [49]:
# This renames the colum headers
df1.columns = ['CPID', 'College', 'PS1']
df2.columns = ['CPID', 'College', 'PS2']

# View the dataframe.  You can move the hashtag to view the other dataframe.
df1
# df2

,CPID,College,PS1
0,3128896,College of Letters and Science,Bojio!\\That was what I playfully typed on my ...
1,3092833,College of Engineering,"My world is shaped by a 5' 10"", lean man, who ..."
2,3142974,College of Engineering,I come from a mediocre family in Malaysia. As ...
3,3020517,College of Letters and Science,Being a Mexican-American now is one of the mos...
4,3121294,College of Natural Resources,I am really confused when I have to fill out i...
5,3162551,College of Letters and Science,"Since before I was born, my parents wanted me ..."
6,3090215,College of Letters and Science,"When I think of my cousin Sid, I see him as th..."
7,3158702,College of Engineering,"I was born in a suburb just east of Seattle, t..."
8,3095465,College of Natural Resources,"Being the youngest in my family, I have been h..."
9,3147750,College of Letters and Science,Living up to the grand expectations of a cardi...


### Merge the Dataframes

Now we will merge the two dataframes on thier two common elements (CPID and College) using `merge`.

In [50]:
# Merge the two data frames so that we have one data frame with both questions attached to common CPID's and College.
df = pandas.merge(df1, df2, on=['CPID', 'College'])
df

,CPID,College,PS1,PS2
0,3128896,College of Letters and Science,Bojio!\\That was what I playfully typed on my ...,Costume: a torn and tattered shirt and pants s...
1,3092833,College of Engineering,"My world is shaped by a 5' 10"", lean man, who ...","Since childhood, I have yearned for utopia. I ..."
2,3142974,College of Engineering,I come from a mediocre family in Malaysia. As ...,I was raised in a family where academics was t...
3,3020517,College of Letters and Science,Being a Mexican-American now is one of the mos...,I learned about the female stereotypes early i...
4,3121294,College of Natural Resources,I am really confused when I have to fill out i...,The bus was loud and smelled. It did not help ...
5,3162551,College of Letters and Science,"Since before I was born, my parents wanted me ...","In the past, when I wanted information and ins..."
6,3090215,College of Letters and Science,"When I think of my cousin Sid, I see him as th...",There's a reason that people buy magazines: th...
7,3158702,College of Engineering,"I was born in a suburb just east of Seattle, t...",I've had a few defining moments throughout my ...
8,3095465,College of Natural Resources,"Being the youngest in my family, I have been h...","One talent that I am proud to have is dancing,..."
9,3147750,College of Letters and Science,Living up to the grand expectations of a cardi...,"Throughout my life, I've explored, enjoyed, an..."


### Review the data

It can be helpful to see how much memory is being used by this new dataframe.  We can do that with the `info` option.  We can also view individual essays, housed in particular cells, in full.

In [51]:
# Check the amount of memory being occupied by this newly created element.  

# PROGRESS
print('Full oringinal dataframe created.')
print(df.info(memory_usage='deep'))

Full oringinal dataframe created.
<class 'pandas.core.frame.DataFrame'>
Int64Index: 82574 entries, 0 to 82573
Data columns (total 4 columns):
CPID       82574 non-null int64
College    82574 non-null object
PS1        82544 non-null object
PS2        82542 non-null object
dtypes: int64(1), object(3)
memory usage: 436.7 MB
None


It is important to review data data that is contained in the new dataframe we created.  This code looks at an essay in full.

In [52]:
#print the first essay from the column 'PS1' the print file is more faithful to our data
print(df['PS1'][0])

Bojio!\\That was what I playfully typed on my family's Whatsapp group chat after my older brother posted a picture of his and my sister in law's Bali resort. It was an expression that travelled from my mind to my flitting fingertips almost immediately. The resort was simply the image of serenity and solitude- and which student going through examination stress would not want to be a part of that?\\It was only when I got home and slumped on the sofa that I saw the nervous look on my mother's face. Her kohl-rimmed eyes were wide and her vermillion adorned forehead scrunched up as she asked, utterly confused, "What's bojio?"\\I burst out laughing. Sometimes, I forgot how every day brought around a new culture shock when you lived in a traditional Indian family but grew up in a multiracial community. The Hokkien phrase "bojio", literally meaning "never invite", is a popular colloquialism in Singapore to teasingly express annoyance at not being invited to something. My brother, who has grown

## 3. Explore the Data using Pandas

Let's first evaluate the general nature of the data to see if the ID's are unique, if there is any missing data, etc.  
We can also look at some descriptive statistics about this data set to get a feel for what's in it. We'll do this using the Pandas package. 

Note: this is always good practice. It serves two purposes. It checks to make sure your data is correct, and there's no major errors. It also keeps you in touch with your data, which will help with interpretation. Love your data!

### Are the ID's Unique?

What ID's have more than one "PS1"s can be found by counting and ranking "ID"s

In [53]:
#This tells us if we have any duplicate IDs.  If each response is 1 we are ok.
print(df['CPID'].value_counts())

# This code seems to check for duplicate CPIDs.  If it's blank there are no duplicates.
print()
print('Array containing duplicate CPIDs:')
print(df.set_index('CPID').index.get_duplicates())

3016703    1
3002500    1
3146453    1
3152598    1
3019479    1
3042008    1
3039961    1
3018728    1
3035871    1
3123936    1
3125987    1
3115748    1
3030135    1
3117799    1
3009256    1
3013354    1
3130093    1
3136238    1
3003119    1
3027667    1
3029714    1
3068623    1
3056321    1
3084983    1
3107512    1
3105465    1
3109563    1
3103422    1
3101375    1
3058368    1
          ..
3044772    1
3014037    1
3040678    1
3042727    1
3151272    1
3153321    1
3012128    1
3132004    1
3028396    1
3007894    1
3132819    1
3048826    1
3132755    1
3134800    1
3061116    1
3059071    1
3136223    1
3100035    1
3110276    1
3021600    1
3085704    1
3130770    1
3081610    1
3144558    1
3093900    1
3095949    1
3089806    1
3091855    1
3132125    1
3116651    1
Name: CPID, dtype: int64

Array containing duplicate CPIDs:
[]


### Find missing essays

Advanced opperations will not work with empty data.  The next few steps are designed to find, exlpore and purge records with missing data.

In [54]:
# This creates a variable empties

# First for PS1
print('Summarizing missing data for PS1:')
empties_PS1 = numpy.where(pandas.isnull(df['PS1']))[0]

print(empties_PS1)

# you notice that this is not formatted as a list.  The next opperation "list" gets it in the right format.
empties_PS1 = list(empties_PS1)
print(empties_PS1)

#This counts the number of missing essays for PS1.
print(len(empties_PS1))

#This lists the elemtns with missing data.
df.iloc[empties_PS1]

Summarizing missing data for PS1:
[ 1776  3206  3566  6285  6801  7530  7930  8111  8571 11796 12977 15694
 19073 23667 24682 26014 28080 28573 29154 29548 31818 40212 41898 44980
 53738 64612 73519 74177 79276 81423]
[1776, 3206, 3566, 6285, 6801, 7530, 7930, 8111, 8571, 11796, 12977, 15694, 19073, 23667, 24682, 26014, 28080, 28573, 29154, 29548, 31818, 40212, 41898, 44980, 53738, 64612, 73519, 74177, 79276, 81423]
30


,CPID,College,PS1,PS2
1776,3133634,College of Engineering,NaN,NaN
3206,3157746,College of Letters and Science,NaN,NaN
3566,3041638,College of Letters and Science,NaN,NaN
6285,3108688,College of Natural Resources,NaN,I never described myself as super religious. W...
6801,3052354,College of Letters and Science,NaN,NaN
7530,3055366,College of Letters and Science,NaN,"At a young age the word ""weird "" or "" differen..."
7930,3056046,College of Engineering,NaN,NaN
8111,3001798,College of Letters and Science,NaN,NaN
8571,3145221,College of Natural Resources,NaN,"The ocean is my world. From an early age, the ..."
11796,3092946,College of Letters and Science,NaN,NaN


In [55]:
# Repeat the above steps for PS2
print('Summarizing missing data for PS2')
empties_PS2 = numpy.where(pandas.isnull(df['PS2']))[0]
print(empties_PS2)
empties_PS2 = list(empties_PS2)
print(empties_PS2)
print(len(empties_PS2))
df.iloc[empties_PS2]

Summarizing missing data for PS2
[ 1222  1776  3206  3566  3667  3900  6801  7930  8111 11796 12977 14552
 16926 19073 23667 24682 26014 28573 31818 36381 40212 41898 44980 53738
 57647 60910 62127 72379 73519 74177 79276 81423]
[1222, 1776, 3206, 3566, 3667, 3900, 6801, 7930, 8111, 11796, 12977, 14552, 16926, 19073, 23667, 24682, 26014, 28573, 31818, 36381, 40212, 41898, 44980, 53738, 57647, 60910, 62127, 72379, 73519, 74177, 79276, 81423]
32


,CPID,College,PS1,PS2
1222,3152303,College of Letters and Science,"Every day, the mirror reminds me that I carry ...",NaN
1776,3133634,College of Engineering,NaN,NaN
3206,3157746,College of Letters and Science,NaN,NaN
3566,3041638,College of Letters and Science,NaN,NaN
3667,3162451,College of Natural Resources,DAD'S BACK ACHED BECAUSE OF ME. Mom's hands mo...,NaN
3900,3060413,College of Letters and Science,"I am lying down on my belly, savoring the exqu...",NaN
6801,3052354,College of Letters and Science,NaN,NaN
7930,3056046,College of Engineering,NaN,NaN
8111,3001798,College of Letters and Science,NaN,NaN
11796,3092946,College of Letters and Science,NaN,NaN


Next we can create a list of every ID which has at least one missing essay.

In [56]:
#This combines the two lists of missing data without duplicateing anything.
empties_any = empties_PS1 + list(set(empties_PS2) - set(empties_PS1))
empties_any.sort()

print(empties_any)
print('There are', len(empties_any), 'CPIDs with at least one missing essay.')

[1222, 1776, 3206, 3566, 3667, 3900, 6285, 6801, 7530, 7930, 8111, 8571, 11796, 12977, 14552, 15694, 16926, 19073, 23667, 24682, 26014, 28080, 28573, 29154, 29548, 31818, 36381, 40212, 41898, 44980, 53738, 57647, 60910, 62127, 64612, 72379, 73519, 74177, 79276, 81423]
There are 40 CPIDs with at least one missing essay.


### Drop the missing essays

This takes the list of ID's that have at least one missing essay and drops them, creating a new dataframe where each cell ID populated.

In [57]:
df_no_missing = df.drop(df.index[empties_any])

# PROGRESS
print('Records with missing data have beend dropped.')

# df_no_missing = df.dropna()
df_no_missing

Records with missing data have beend dropped.


,CPID,College,PS1,PS2
0,3128896,College of Letters and Science,Bojio!\\That was what I playfully typed on my ...,Costume: a torn and tattered shirt and pants s...
1,3092833,College of Engineering,"My world is shaped by a 5' 10"", lean man, who ...","Since childhood, I have yearned for utopia. I ..."
2,3142974,College of Engineering,I come from a mediocre family in Malaysia. As ...,I was raised in a family where academics was t...
3,3020517,College of Letters and Science,Being a Mexican-American now is one of the mos...,I learned about the female stereotypes early i...
4,3121294,College of Natural Resources,I am really confused when I have to fill out i...,The bus was loud and smelled. It did not help ...
5,3162551,College of Letters and Science,"Since before I was born, my parents wanted me ...","In the past, when I wanted information and ins..."
6,3090215,College of Letters and Science,"When I think of my cousin Sid, I see him as th...",There's a reason that people buy magazines: th...
7,3158702,College of Engineering,"I was born in a suburb just east of Seattle, t...",I've had a few defining moments throughout my ...
8,3095465,College of Natural Resources,"Being the youngest in my family, I have been h...","One talent that I am proud to have is dancing,..."
9,3147750,College of Letters and Science,Living up to the grand expectations of a cardi...,"Throughout my life, I've explored, enjoyed, an..."


Now we can re-index the essays so we have a consistent count.

In [58]:
df_no_missing = df_no_missing.reset_index(drop=True)
df_no_missing

,CPID,College,PS1,PS2
0,3128896,College of Letters and Science,Bojio!\\That was what I playfully typed on my ...,Costume: a torn and tattered shirt and pants s...
1,3092833,College of Engineering,"My world is shaped by a 5' 10"", lean man, who ...","Since childhood, I have yearned for utopia. I ..."
2,3142974,College of Engineering,I come from a mediocre family in Malaysia. As ...,I was raised in a family where academics was t...
3,3020517,College of Letters and Science,Being a Mexican-American now is one of the mos...,I learned about the female stereotypes early i...
4,3121294,College of Natural Resources,I am really confused when I have to fill out i...,The bus was loud and smelled. It did not help ...
5,3162551,College of Letters and Science,"Since before I was born, my parents wanted me ...","In the past, when I wanted information and ins..."
6,3090215,College of Letters and Science,"When I think of my cousin Sid, I see him as th...",There's a reason that people buy magazines: th...
7,3158702,College of Engineering,"I was born in a suburb just east of Seattle, t...",I've had a few defining moments throughout my ...
8,3095465,College of Natural Resources,"Being the youngest in my family, I have been h...","One talent that I am proud to have is dancing,..."
9,3147750,College of Letters and Science,Living up to the grand expectations of a cardi...,"Throughout my life, I've explored, enjoyed, an..."


## 4. Pre-Processing the Essays

Once we have a Pandas Dataframe in the appropreiate structure, we can begin to process the text in the two essay columns.  This invovles cleaning the text in a number of ways before tokenizing the essays.  

### Cleaning the text and tokenizing

The section below combines multiple preprocessing steps into a singl eline of code.  It is repeated twice for each of the essays, and results in a largely "preprocessed", tokenized new column.  most of this is accomplished with the "str" feature of python.  Here is what we accomplished with each step:
1. `str.replace('\\', ' ')` - This removes some of the ideosyncratic backslashes that were present 
1. `str.lower()` - this shifts all the letters to lowercase
1. `str.replace('[^\w\s]','')` - This gets rid of punctuation.  The "`^`" is a negated set, the "`\w`" matches any word character (alphanumeric & underscore), and the "`\s`" matches any whitespace character (spaces, tabs, line breaks).
1. `str.replace('[\d]','')` - This gets rid of all numbers.
1. `str.split()` - This tokenizes whats left, creating a list within the pandas cell

In [59]:
#create two new columns with tokenized essay responses.  
#In the same opperation it make everything lowercase.
df_no_missing['PS1_clean'] = df_no_missing['PS1'].str.replace('\\', ' ').str.lower().str.replace('[^\w\s]','').str.replace('[\d]','').str.split()
df_no_missing['PS2_clean'] = df_no_missing['PS2'].str.replace('\\', ' ').str.lower().str.replace('[^\w\s]','').str.replace('[\d]','').str.split()

# PROGRESS
print('Removed weird characters and creats tokens in two new columns.')

df_no_missing

Removes weird charactors and creats tokens in two new columns.


,CPID,College,PS1,PS2,PS1_clean,PS2_clean
0,3128896,College of Letters and Science,Bojio!\\That was what I playfully typed on my ...,Costume: a torn and tattered shirt and pants s...,"[bojio, that, was, what, i, playfully, typed, ...","[costume, a, torn, and, tattered, shirt, and, ..."
1,3092833,College of Engineering,"My world is shaped by a 5' 10"", lean man, who ...","Since childhood, I have yearned for utopia. I ...","[my, world, is, shaped, by, a, lean, man, who,...","[since, childhood, i, have, yearned, for, utop..."
2,3142974,College of Engineering,I come from a mediocre family in Malaysia. As ...,I was raised in a family where academics was t...,"[i, come, from, a, mediocre, family, in, malay...","[i, was, raised, in, a, family, where, academi..."
3,3020517,College of Letters and Science,Being a Mexican-American now is one of the mos...,I learned about the female stereotypes early i...,"[being, a, mexicanamerican, now, is, one, of, ...","[i, learned, about, the, female, stereotypes, ..."
4,3121294,College of Natural Resources,I am really confused when I have to fill out i...,The bus was loud and smelled. It did not help ...,"[i, am, really, confused, when, i, have, to, f...","[the, bus, was, loud, and, smelled, it, did, n..."
5,3162551,College of Letters and Science,"Since before I was born, my parents wanted me ...","In the past, when I wanted information and ins...","[since, before, i, was, born, my, parents, wan...","[in, the, past, when, i, wanted, information, ..."
6,3090215,College of Letters and Science,"When I think of my cousin Sid, I see him as th...",There's a reason that people buy magazines: th...,"[when, i, think, of, my, cousin, sid, i, see, ...","[theres, a, reason, that, people, buy, magazin..."
7,3158702,College of Engineering,"I was born in a suburb just east of Seattle, t...",I've had a few defining moments throughout my ...,"[i, was, born, in, a, suburb, just, east, of, ...","[ive, had, a, few, defining, moments, througho..."
8,3095465,College of Natural Resources,"Being the youngest in my family, I have been h...","One talent that I am proud to have is dancing,...","[being, the, youngest, in, my, family, i, have...","[one, talent, that, i, am, proud, to, have, is..."
9,3147750,College of Letters and Science,Living up to the grand expectations of a cardi...,"Throughout my life, I've explored, enjoyed, an...","[living, up, to, the, grand, expectations, of,...","[throughout, my, life, ive, explored, enjoyed,..."


In [60]:
# this shows that we've mostly delt with the odd backslashes and cleand the text in a bunch of other ways!
print(df_no_missing['PS1_clean'][0])

['bojio', 'that', 'was', 'what', 'i', 'playfully', 'typed', 'on', 'my', 'familys', 'whatsapp', 'group', 'chat', 'after', 'my', 'older', 'brother', 'posted', 'a', 'picture', 'of', 'his', 'and', 'my', 'sister', 'in', 'laws', 'bali', 'resort', 'it', 'was', 'an', 'expression', 'that', 'travelled', 'from', 'my', 'mind', 'to', 'my', 'flitting', 'fingertips', 'almost', 'immediately', 'the', 'resort', 'was', 'simply', 'the', 'image', 'of', 'serenity', 'and', 'solitude', 'and', 'which', 'student', 'going', 'through', 'examination', 'stress', 'would', 'not', 'want', 'to', 'be', 'a', 'part', 'of', 'that', 'it', 'was', 'only', 'when', 'i', 'got', 'home', 'and', 'slumped', 'on', 'the', 'sofa', 'that', 'i', 'saw', 'the', 'nervous', 'look', 'on', 'my', 'mothers', 'face', 'her', 'kohlrimmed', 'eyes', 'were', 'wide', 'and', 'her', 'vermillion', 'adorned', 'forehead', 'scrunched', 'up', 'as', 'she', 'asked', 'utterly', 'confused', 'whats', 'bojio', 'i', 'burst', 'out', 'laughing', 'sometimes', 'i', 'for

### Remove Stopwords

Stopwords are  words that appear frequently and tend not to be distinctive.  They are generally removed prior to text analysis unless there is compelling reason to keep them. [More info](http://www.nltk.org/book/ch02.html#code-unusual)

In [61]:
#stopwords imported from NLTK Above

#Removes english stop words from the tokenized columns
stop_words = stopwords.words('english')
df_no_missing['PS1_clean'] = df_no_missing['PS1_clean'].apply(lambda x: [item for item in x if item not in stop_words])
df_no_missing['PS2_clean'] = df_no_missing['PS2_clean'].apply(lambda x: [item for item in x if item not in stop_words])

# PROGRESS
print('Stopwords have been removed.')

df_no_missing

Stopwords have been removed.


,CPID,College,PS1,PS2,PS1_clean,PS2_clean
0,3128896,College of Letters and Science,Bojio!\\That was what I playfully typed on my ...,Costume: a torn and tattered shirt and pants s...,"[bojio, playfully, typed, familys, whatsapp, g...","[costume, torn, tattered, shirt, pants, old, c..."
1,3092833,College of Engineering,"My world is shaped by a 5' 10"", lean man, who ...","Since childhood, I have yearned for utopia. I ...","[world, shaped, lean, man, looks, like, despit...","[since, childhood, yearned, utopia, know, appe..."
2,3142974,College of Engineering,I come from a mediocre family in Malaysia. As ...,I was raised in a family where academics was t...,"[come, mediocre, family, malaysia, malaysian, ...","[raised, family, academics, number, one, prior..."
3,3020517,College of Letters and Science,Being a Mexican-American now is one of the mos...,I learned about the female stereotypes early i...,"[mexicanamerican, one, amazing, things, could,...","[learned, female, stereotypes, early, life, wo..."
4,3121294,College of Natural Resources,I am really confused when I have to fill out i...,The bus was loud and smelled. It did not help ...,"[really, confused, fill, information, comes, s...","[bus, loud, smelled, help, ac, yellow, school,..."
5,3162551,College of Letters and Science,"Since before I was born, my parents wanted me ...","In the past, when I wanted information and ins...","[since, born, parents, wanted, identify, dutch...","[past, wanted, information, inspiration, knew,..."
6,3090215,College of Letters and Science,"When I think of my cousin Sid, I see him as th...",There's a reason that people buy magazines: th...,"[think, cousin, sid, see, guy, loves, nascar, ...","[theres, reason, people, buy, magazines, want,..."
7,3158702,College of Engineering,"I was born in a suburb just east of Seattle, t...",I've had a few defining moments throughout my ...,"[born, suburb, east, seattle, middle, class, f...","[ive, defining, moments, throughout, life, cha..."
8,3095465,College of Natural Resources,"Being the youngest in my family, I have been h...","One talent that I am proud to have is dancing,...","[youngest, family, held, much, higher, standar...","[one, talent, proud, dancing, dancing, since, ..."
9,3147750,College of Letters and Science,Living up to the grand expectations of a cardi...,"Throughout my life, I've explored, enjoyed, an...","[living, grand, expectations, cardiologistprof...","[throughout, life, ive, explored, enjoyed, cha..."


### Stem the Tokens

Stemming reduces words with multiple endings to thier common stem.  There are multiple ways to do this, but we will use the Porter Stemmer for our purpouses. http://www.bogotobogo.com/python/NLTK/Stemming_NLTK.php

In [62]:
# using the "Porter Stemmer" we'll stem the words
porter_stemmer = PorterStemmer()

# There is lots of code in the saved file to rework this if nessisary.

#This works on my machine with NLTK 3.2.1, but not on Mercury when it had NLTK 3.2.2!
df_no_missing['PS1_clean'] = df_no_missing['PS1_clean'].apply(lambda x: [porter_stemmer.stem(item) for item in x])
df_no_missing['PS2_clean'] = df_no_missing['PS2_clean'].apply(lambda x: [porter_stemmer.stem(item) for item in x])

# PROGRESS
print('Stemmed the tokens.')

df_no_missing_tokens = df_no_missing
df_no_missing

Stemmed the tokens.


,CPID,College,PS1,PS2,PS1_clean,PS2_clean
0,3128896,College of Letters and Science,Bojio!\\That was what I playfully typed on my ...,Costume: a torn and tattered shirt and pants s...,"[bojio, play, type, famili, whatsapp, group, c...","[costum, torn, tatter, shirt, pant, old, colou..."
1,3092833,College of Engineering,"My world is shaped by a 5' 10"", lean man, who ...","Since childhood, I have yearned for utopia. I ...","[world, shape, lean, man, look, like, despit, ...","[sinc, childhood, yearn, utopia, know, appear,..."
2,3142974,College of Engineering,I come from a mediocre family in Malaysia. As ...,I was raised in a family where academics was t...,"[come, mediocr, famili, malaysia, malaysian, e...","[rais, famili, academ, number, one, prioriti, ..."
3,3020517,College of Letters and Science,Being a Mexican-American now is one of the mos...,I learned about the female stereotypes early i...,"[mexicanamerican, one, amaz, thing, could, eve...","[learn, femal, stereotyp, earli, life, women, ..."
4,3121294,College of Natural Resources,I am really confused when I have to fill out i...,The bus was loud and smelled. It did not help ...,"[realli, confus, fill, inform, come, standard,...","[bu, loud, smell, help, ac, yellow, school, bu..."
5,3162551,College of Letters and Science,"Since before I was born, my parents wanted me ...","In the past, when I wanted information and ins...","[sinc, born, parent, want, identifi, dutch, am...","[past, want, inform, inspir, knew, look, class..."
6,3090215,College of Letters and Science,"When I think of my cousin Sid, I see him as th...",There's a reason that people buy magazines: th...,"[think, cousin, sid, see, guy, love, nascar, a...","[there, reason, peopl, buy, magazin, want, lik..."
7,3158702,College of Engineering,"I was born in a suburb just east of Seattle, t...",I've had a few defining moments throughout my ...,"[born, suburb, east, seattl, middl, class, fam...","[ive, defin, moment, throughout, life, chang, ..."
8,3095465,College of Natural Resources,"Being the youngest in my family, I have been h...","One talent that I am proud to have is dancing,...","[youngest, famili, held, much, higher, standar...","[one, talent, proud, danc, danc, sinc, kinderg..."
9,3147750,College of Letters and Science,Living up to the grand expectations of a cardi...,"Throughout my life, I've explored, enjoyed, an...","[live, grand, expect, cardiologistprofessor, n...","[throughout, life, ive, explor, enjoy, challen..."


### Testing token counts

We may want to drop not only blank essays, but essays are very short under the assumption that they were not completed or not intended as full essays.  We should also consider droping particularly short essays after they have been stemmed, which can subsantially cut down on the number words available for analysis.

For now, these are not removed.

In [65]:
post_processing_token_nan_1 = (df_no_missing_tokens['PS1_clean'].str.len() < 10)
post_processing_token_nan_1

post_processing_token_nan_df_1 = df_no_missing.loc[post_processing_token_nan_1]
post_processing_token_nan_df_1

,CPID,College,PS1,PS2,PS1_clean,PS2_clean
471,3041105,College of Letters and Science,The world I came from is a pretty peaceful wor...,One of my best qualities is how I'm very atten...,"[world, came, pretti, peac, world, chang, drea...","[one, best, qualiti, im, attent, everyth, arou..."
1229,3015017,College of Engineering,I am Arabian a have 5 brothers and 5 sisters I...,"I was working at special forces, I graduated f...","[arabian, brother, sister, finish, high, school]","[work, special, forc, graduat, antiterror, col..."
6843,3166277,College of Chemistry,-,"Cold, heavy rain drops fell onto my face. Some...",[],"[cold, heavi, rain, drop, fell, onto, face, so..."
12829,3155188,College of Engineering,People always ask my parents how me and my sis...,"While growing up, I was never the biggest, fas...","[peopl, alway, ask, parent, sister, well, beha...","[grow, never, biggest, fastest, outgo, even, t..."
14476,3084615,College of Letters and Science,A,B,[],[b]
18641,3111191,College of Letters and Science,.,"My dad once said, ""Someone can take your wealt...",[],"[dad, said, someon, take, wealth, properti, on..."
21781,3120546,College of Engineering,The world I come from is,"Life is full of conflict, however the problems...","[world, come]","[life, full, conflict, howev, problem, encount..."
26490,3161087,College of Letters and Science,Its all said in Essay 2,"""Sit and be quiet,"" my mom would hush as I sat...","[said, essay]","[sit, quiet, mom, would, hush, sat, cold, unco..."
27657,3032667,College of Letters and Science,Recruited student-athlete,Recruited student-athlete,"[recruit, studentathlet]","[recruit, studentathlet]"
30340,3161598,College of Chemistry,Ms word doc,Ms word doc,"[ms, word, doc]","[ms, word, doc]"


In [76]:
print(df_no_missing_tokens['PS1'][471])
print(df_no_missing_tokens['PS1_clean'][471])

print(df_no_missing_tokens['PS1'][1229])
print(df_no_missing_tokens['PS1_clean'][1229])

print(df_no_missing_tokens['PS1'][12829])
print(df_no_missing_tokens['PS1_clean'][12829])

print(df_no_missing_tokens['PS1'][21781])
print(df_no_missing_tokens['PS1_clean'][21781])

print(df_no_missing_tokens['PS1'][26490])
print(df_no_missing_tokens['PS1_clean'][26490])

# TO DO: write a script that gets captures these id's as a list and loops through them to print.


The world I came from is a pretty peaceful world, it has not changed any dreams of mine.
['world', 'came', 'pretti', 'peac', 'world', 'chang', 'dream', 'mine']
I am Arabian a have 5 brothers and 5 sisters I finished my high school in 2002/2003.
['arabian', 'brother', 'sister', 'finish', 'high', 'school']
People always ask my parents how me and my sister are so well behaved, and they can with all honesty
['peopl', 'alway', 'ask', 'parent', 'sister', 'well', 'behav', 'honesti']
The world I come from is
['world', 'come']
Its all said in Essay 2
['said', 'essay']
I am Arabian a have 5 brothers and 5 sisters I finished my high school in 2002/2003.
['arabian', 'brother', 'sister', 'finish', 'high', 'school']
I am Arabian a have 5 brothers and 5 sisters I finished my high school in 2002/2003.
['arabian', 'brother', 'sister', 'finish', 'high', 'school']
I am Arabian a have 5 brothers and 5 sisters I finished my high school in 2002/2003.
['arabian', 'brother', 'sister', 'finish', 'high', 'schoo

In [40]:
post_processing_token_nan_2 = (df_no_missing['PS2_clean'].str.len() < 20)
post_processing_token_nan_2

post_processing_token_nan_df_2 = df_no_missing.loc[post_processing_token_nan_2]
post_processing_token_nan_df_2

,CPID,College,PS1,PS2,PS1_clean,PS2_clean
2693,3094431,College of Letters and Science,I became interested in Economics four years ag...,a,becam interest econom four year ago read krugm...,
4454,3154717,College of Letters and Science,I am horrible at making decisions. In the begi...,.,horribl make decis begin high school embarrass...,
6531,3119005,College of Letters and Science,Tajikistan is a small country in Central Asia....,see1,tajikistan small countri central asia place ta...,see
14476,3084615,College of Letters and Science,A,B,,b
22047,3102346,College of Letters and Science,Well to start off the world I come from hasn't...,Cross Country,well start world come hasnt exactli smooth jou...,cross countri
22210,3093771,College of Letters and Science,Many people experience hard times or tragic ev...,One experience that is important to me,mani peopl experi hard time tragic event span ...,one experi import
26994,3022238,College of Natural Resources,My name is Zahra Hejazi.I live in Iran with my...,i really like studying,name zahra hejazii live iran parent littl brot...,realli like studi
30183,3119489,College of Letters and Science,"College Admission Essay\November 15, 2015\\Col...",.,colleg admiss essay novemb colleg admiss essay...,
30340,3161598,College of Chemistry,Ms word doc,Ms word doc,ms word doc,ms word doc
31182,3009014,College of Natural Resources,xxxx,xxxx,xxxx,xxxx


### Convert tokens back to strings

Joining the tokens back to a string so we can execute count vectorizer and create a documnet term matrix.

In [20]:
df_no_missing['PS1_clean'] = df_no_missing['PS1_clean'].apply(lambda x: ' '.join(x)) # for item in x])
df_no_missing['PS2_clean'] = df_no_missing['PS2_clean'].apply(lambda x: ' '.join(x)) # for item in x])

# PROGRESS
print('Transformed the tokens back into strings.')

# text_list_stemmed = [' '.join([porter_stemmer.stem(word) for word in sentence.split(" ")]) for sentence in text_list]
df_no_missing

Transformed the tokens back into strings.


,CPID,College,PS1,PS2,PS1_clean,PS2_clean
0,3128896,College of Letters and Science,Bojio!\\That was what I playfully typed on my ...,Costume: a torn and tattered shirt and pants s...,bojio play type famili whatsapp group chat old...,costum torn tatter shirt pant old colour fade ...
1,3092833,College of Engineering,"My world is shaped by a 5' 10"", lean man, who ...","Since childhood, I have yearned for utopia. I ...",world shape lean man look like despit age look...,sinc childhood yearn utopia know appear unreal...
2,3142974,College of Engineering,I come from a mediocre family in Malaysia. As ...,I was raised in a family where academics was t...,come mediocr famili malaysia malaysian expos d...,rais famili academ number one prioriti excel g...
3,3020517,College of Letters and Science,Being a Mexican-American now is one of the mos...,I learned about the female stereotypes early i...,mexicanamerican one amaz thing could ever fina...,learn femal stereotyp earli life women weak ca...
4,3121294,College of Natural Resources,I am really confused when I have to fill out i...,The bus was loud and smelled. It did not help ...,realli confus fill inform come standard test s...,bu loud smell help ac yellow school bu transpo...
5,3162551,College of Letters and Science,"Since before I was born, my parents wanted me ...","In the past, when I wanted information and ins...",sinc born parent want identifi dutch american ...,past want inform inspir knew look classroom te...
6,3090215,College of Letters and Science,"When I think of my cousin Sid, I see him as th...",There's a reason that people buy magazines: th...,think cousin sid see guy love nascar alway ask...,there reason peopl buy magazin want like way l...
7,3158702,College of Engineering,"I was born in a suburb just east of Seattle, t...",I've had a few defining moments throughout my ...,born suburb east seattl middl class famili two...,ive defin moment throughout life chang futur o...
8,3095465,College of Natural Resources,"Being the youngest in my family, I have been h...","One talent that I am proud to have is dancing,...",youngest famili held much higher standard olde...,one talent proud danc danc sinc kindergarten f...
9,3147750,College of Letters and Science,Living up to the grand expectations of a cardi...,"Throughout my life, I've explored, enjoyed, an...",live grand expect cardiologistprofessor nutrit...,throughout life ive explor enjoy challeng inte...


This creates an alternate dataframe that can be used to try to drop short entries after stemming.

In [32]:
# df_no_missing
#post_processing_nan = (df_no_missing['PS1'].str.len() < 20) ^ (df_no_missing['PS2'].str.len() < 20) ^ (df_no_missing['PS1_clean'].str.len() < 20) ^ (df_no_missing['PS2_clean'].str.len() < 20)

#post_processing_nan = (df_no_missing['PS1'].str.len() < 20)
post_processing_nan = (df_no_missing['PS2'].str.len() < 20)
#post_processing_nan = (df_no_missing['PS1_clean'].str.len() < 20)
#post_processing_nan = (df_no_missing['PS2_clean'].str.len() < 20)

doc_topic_df_short = df_no_missing.loc[post_processing_nan]
doc_topic_df_short

,CPID,College,PS1,PS2,PS1_clean,PS2_clean
2695,3094431,College of Letters and Science,I became interested in Economics four years ag...,a,becam interest econom four year ago read krugm...,
4460,3154717,College of Letters and Science,I am horrible at making decisions. In the begi...,.,horribl make decis begin high school embarrass...,
6538,3119005,College of Letters and Science,Tajikistan is a small country in Central Asia....,see1,tajikistan small countri central asia place ta...,see
14490,3084615,College of Letters and Science,A,B,,b
22065,3102346,College of Letters and Science,Well to start off the world I come from hasn't...,Cross Country,well start world come hasnt exactli smooth jou...,cross countri
30208,3119489,College of Letters and Science,"College Admission Essay\November 15, 2015\\Col...",.,colleg admiss essay novemb colleg admiss essay...,
30365,3161598,College of Chemistry,Ms word doc,Ms word doc,ms word doc,ms word doc
31207,3009014,College of Natural Resources,xxxx,xxxx,xxxx,xxxx
40726,3115474,College of Letters and Science,"A Better Me\As Paul Day said, ""All separation ...",All in Statement 1,better paul day said separ involv suspend mome...,statement
48476,3153233,College of Natural Resources,I hated trying out new food. The mystery of th...,asdasddasd,hate tri new food mysteri new dish potenti dan...,asdasddasd


## 5. Creating a sample for testing

In this section we'll create a smaller sample of the code to that the analysis we construct below works.

In [33]:
# This generates a random sample of N essays, with a random state set for reproducability.
# N can be slowly increased slowly to test the computational resources required as you scale up
df_sample = df_no_missing.sample(n=500, random_state=0)

# This code resets the indexs so that sorted orininals are kept and new ones are generated.
df_sample = df_sample.sort_index()
df_sample = df_sample.reset_index()

# PROGRESS
print('Created a sample of the data.')

#This is where I assign the sample data to be analyzed.  If I want to run the whole dataset, comment this out.
#df_no_missing = df_sample

if (sample_yes_no == 'Y') or (sample_yes_no == 'YES'):
    df_no_missing = df_sample
    print('Running on the sample, not the whole dataset.')
elif (sample_yes_no == 'N') or (sample_yes_no == 'NO'):
    print('Running on the entire dataset.')
else:
    print('Unable to determine what data to analyze, the sample or the entire set.')

df_sample

Created a sample of the data.
Running on the entire dataset.


,index,CPID,College,PS1,PS2,PS1_clean,PS2_clean
0,338,3163703,College of Natural Resources,"As I stared into the vulgar man's eyes, I felt...",Living in a society that praises women for the...,stare vulgar man eye felt nail dig deeper fles...,live societi prais women extern beauti instead...
1,439,3056579,College of Engineering,I come from an Indian family whose ancestry tr...,My involvement in speech and debate has had a ...,come indian famili whose ancestri trace back i...,involv speech debat signific impact ive becom ...
2,722,3113770,College of Letters and Science,Whenever I walked into my mother's room at the...,I was part of a student volunteer project to e...,whenev walk mother room begin month would ofte...,part student volunt project encourag physic me...
3,794,3120168,College of Engineering,The early years of my childhood in Mexico were...,My life is one of numbers. The universal langu...,earli year childhood mexico easi fact first ye...,life one number univers languag math transcend...
4,822,3150465,College of Letters and Science,My social life in reality was not exactly stel...,"A few months ago, I had an experience that sho...",social life realiti exactli stellar start foun...,month ago experi show much leadership skill im...
5,1102,3047627,College of Chemistry,Most people acquire a unique personality based...,"A human's natural instinct is to fear failure,...",peopl acquir uniqu person base upon individu c...,human natur instinct fear failur go life avoid...
6,1302,3094299,College of Letters and Science,"First, I noticed her trembling hands. In the y...","I knocked tentatively on her door, fidgeting u...",first notic trembl hand year known never seen ...,knock tent door fidget uneasili muffl cri echo...
7,1467,3112108,College of Natural Resources,Sophomore year I decided to transfer to Campbe...,"In 2nd grade, I joined Cub Scouts with a few o...",sophomor year decid transfer campbel hall desi...,nd grade join cub scout friend love outdoor en...
8,1478,3129573,College of Engineering,"Every year, distant relatives gather at our ho...",I've always looked forward to the day I'd fini...,everi year distant rel gather hous celebr call...,ive alway look forward day id finish middl sch...
9,1563,3042091,College of Letters and Science,I am the youngest of three brothers and come f...,"As a high school senior, I find that the most ...",youngest three brother come famili known deter...,high school senior find defin qualiti famili p...


## 6. Creating the DTM: scikit-learn

Now that we've preprocessed the text and created two colums with strings, the required imput for scikit-learn's CountVectorizer, we can create a documnet term matrix.  This is the building block for Topic Modeling and a number of other methods we may want to explore.  There are two ways to do this. We can turn it into a sparse matrix type, which can be used within scikit-learn for further analyses.  We can then turn it into a full documnet term matrix, but this is very memory intensive and might not be a great idea for larger data sets.

In [ ]:
# see above for: from sklearn.feature_extraction.text import CountVectorizer
countvec = CountVectorizer()

#Original sklearn_dtm = CountVectorizer().fit_transform(df.PS1)
#I added the '.values.astype('U')' for an interim step in the section below. 
#It seemed to fix the count vectorizer issues
sklearn_dtm_PS1 = countvec.fit_transform(df_no_missing['PS1_clean'])
sklearn_dtm_PS2 = countvec.fit_transform(df_no_missing['PS2_clean'])

print('PS1 sparse matrix type')
print(sklearn_dtm_PS1)
print(' ')
print('PS2 sparse matrix type')
print(sklearn_dtm_PS2)

## 7. Tf-idf scores

For future applicnation:

How to find distinctive words in a corpus is a long-standing question in text analysis. The idea behind words scores is to weight words not just by their frequency, but by their frequency in one document compared to their distribution across all documents. Words that are frequent, but are also used in every single document, will not be distinguising. We want to identify words that are unevenly distributed across the corpus.

One of the most popular ways to weight words (beyond frequency counts) is *tf-idf* scores. By offsetting the frequency of a word by its document frequency (the number of documents in which it appears) will in theory filter out common terms such as 'the', 'of', and 'and'.

We can calculate this manually, but scikit-learn has a built-in function to do so. We'll use it, but a challenge for you: use Pandas to calculate this manually. 

To do so, we simply do the same thing we did above with CountVectorizer, but instead we use the function TfidfVectorizer.

In [ ]:
# see above for from sklearn.feature_extraction.text import TfidfVectorizer
tfidfvec = TfidfVectorizer()

# #create the dtm, but with cells weigthed by the tf-idf score.
# dtm_tfidf_df = pandas.DataFrame(tfidfvec.fit_transform(df.PS1).toarray(), columns=tfidfvec.get_feature_names(), index = df.index)

# #view results
# dtm_tfidf_df

Let's look at the 20 words with highest tf-idf weights.

In [ ]:
# print(dtm_tfidf_df.max().sort_values(ascending=False)[0:20])

## 8. Uncovering Patterns: LDA

Frequency counts and tf-idf scores are done at the word level. There are other methods of exporatory or unsupervised analysis on the document level and by examining the co-occurrence of words within documents. Scikit-learn allows for many of these methods, including:

* document clustering
* document or word similarities using cosine similarity
* pca
* topic modeling

We'll use topic modeling here. 

We will run Latent Dirichlet Allocation, the most basic and the oldest version of topic modeling. We will run this in one big chunk of code. 

In [ ]:
# df_lit = pandas.read_csv("/Volumes/Extra Space/Google Drive/Scholarship/Writing Projects - Personal/Admissions Essays/Small Sample/AdmissionsEssays/statement_test_031417.csv", sep = ',', encoding = 'utf-8')

# #drop rows where the text is missing. I think there's only one row where it's missing, but check me on that.
# df_lit = df_lit.dropna(subset=['PS1'])

#df_lit = df_no_missing

#view the dataframe
#df_lit

Now we're ready to fit the model. This requires the use of CountVecorizer, which we've already used, and the scikit-learn function LatentDirichletAllocation.

See [here](http://scikit-learn.org/stable/modules/generated/sklearn.decomposition.LatentDirichletAllocation.html) for more information about this function. 

### Running the LDA Model

In [ ]:
#should switch to batch (from online).  n_samples should be closer to full set

####Adopted From: 
#Author: Olivier Grisel <olivier.grisel@ensta.org>
#         Lars Buitinck
#         Chyi-Kwei Yau <chyikwei.yau@gmail.com>
# License: BSD 3 clause

# See above for: from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
# and:from sklearn.decomposition import LatentDirichletAllocation

n_samples = 2000
n_topics = number_of_topics #changed this so the value can be set above.
n_top_words = 100

##This is a function to print out the top words for each topic in a pretty way.
#Don't worry too much about understanding every line of this code.
def print_top_words(model, feature_names, n_top_words):
    for topic_idx, topic in enumerate(model.components_):
        print("\nTopic #%d:" % topic_idx)
        print(" ".join([feature_names[i]
                        for i in topic.argsort()[:-n_top_words - 1:-1]]))
    print()

# Use tf-idf features
tfidf_vectorizer = TfidfVectorizer(max_df=0.80, min_df=50,
                                   max_features=None,
                                   stop_words='english')

tfidf = tfidf_vectorizer.fit_transform(df_no_missing['PS1_clean'])

# Use tf (raw term count) features
print("Extracting tf features for LDA...")
tf_vectorizer = CountVectorizer(max_df=0.80, min_df=50,
                                max_features=None,
                                stop_words='english'
                                )

tf = tf_vectorizer.fit_transform(df_no_missing['PS1_clean'])

print("Fitting LDA models with tf features, "
      "n_samples=%d and n_topics=%d..."
      % (n_samples, n_topics))

#define the lda function, with desired options TAKE A LOOK AT THIS.  MIGHT BE TOO FEW
lda = LatentDirichletAllocation(n_topics=n_topics, max_iter=100,  #100 (THATS WHAT LAURA DID)
                                learning_method='batch',  # CHANGE THIS from 'online' TO 'batch'
                                learning_offset=80.,
                                total_samples=n_samples,
                                random_state=0)
#fit the model
lda.fit(tf)

#print the top words per topic, using the function defined above.
#Unlike R, which has a built-in function to print top words, we have to write our own for scikit-learn
#I think this demonstrates the different aims of the two packages: R is for social scientists, Python for computer scientists

print("\nTopics in LDA model:")
tf_feature_names = tf_vectorizer.get_feature_names()
print_top_words(lda, tf_feature_names, n_top_words)

### Format Topic Document Matrix

One thing we may want to do with the output is find the most representative texts for each topic. A simple way to do this (but not memory efficient), is to merge the topic distribution back into the Pandas dataframe.

First get the topic distribution array. The numbers produced here sum to 1 for every row, and represent the relative representativeness of each topic.  The topic with the highest number is the most likely topic to have structured the documnemt.  

In [ ]:
topic_dist = lda.transform(tf)
topic_dist

Then we create a Panda's Dataframe and do some work to rename the columns.

In [ ]:
#this reads the array into a dataframe
topic_dist_df = pandas.DataFrame(topic_dist)

#This is a little script to name the colum names appropriately
col_number = n_topics
col_nubber_original = col_number
column_name_string_PS1 = ""
for i in range(0,col_number):
    column_name_string_PS1+="'Topic_"+str(col_nubber_original-col_number)+"_PS1', "
    col_number = col_number - 1


#This cleans up the end of the string produced above and adds brackets
column_name_string_PS1 = column_name_string_PS1[:-2]
column_name_string_PS1 = '['+column_name_string_PS1+']'


#This converts the string into a list type object.
column_name_string_PS1 = ast.literal_eval(column_name_string_PS1)

#This applies the string contining the names of the colums to the dataframe
topic_dist_df.columns = column_name_string_PS1

topic_dist_df

Now we can merge the new dataframe with the original dataframe of essays and ID's.

In [ ]:
df_w_topics = df_no_missing.join(topic_dist_df)

# PROGRESS
print('Joined the topic and oringinal dataframes.')

df_w_topics

### Save the Documnet Topic Matrix

In [ ]:
#Writing the output of this run to a CSV
#df_w_topics.to_csv('Admissions_PS1_Full_'+time_for_f_name+'.csv', sep=',')

path = '../data/scratch_dir/'
topic_num_string = str(number_of_topics)

#print(path+'Admissions_PS1_Full_'+time_for_f_name+'.csv')

# add this   number_of_topics   and test it: 
df_w_topics.to_csv(path+'Admissions_PS1_'+topic_num_string+'_Topic_'+time_for_f_name+'.gzip', compression='gzip', sep=',')

### Format Feature Matrix

These first few steps walk through the data that we have and tries to get it into a format that is readable.  First, we take a look at a list of words stored in tf_feautre_names.

In [ ]:
print(len(tf_feature_names))
print(type(tf_feature_names))
print(tf_feature_names)

We are going to need to transform the list into an array, which we can do below.

In [ ]:
# This transforms the list into a numpy array
tf_feature_names_array = numpy.asarray(tf_feature_names)
print(type(tf_feature_names_array))
print(tf_feature_names_array.shape)
print(tf_feature_names_array)

This is a bit of an asside, but we see that this matrix stores the topic weights for each document.

In [ ]:
#These identical arrays provide the topic weighting for each text.
print(lda.transform(tf))
print(lda.transform(tf).shape)
print(topic_dist)
print(topic_dist.shape)

Now we can look at a matrix that seems to store the feature weights.  We notice that the dimentions of this martix correspond to the number of topics and the dimentions of the word list we just reviewed above.

In [ ]:
#These two things tell us what is in the LDA model at this point
#I believe this tell us the weights of the features, which in this case are words

#this tells us the dimentions of the array
print(lda.components_.shape)
print(type(lda.components_))

#This shows that components is an array of numbers
print(lda.components_)

This transposes the matrix so it matches the terms.

In [ ]:
transposed_components = numpy.transpose(lda.components_)
print(transposed_components.shape)
print(type(transposed_components))
print(transposed_components)

Now we can conver the feature list into a Pandas DataFrame for easy viewing and manipulation.

In [ ]:
item_new = pandas.DataFrame(tf_feature_names_array)
item_new.columns = ['Feature_Words']
item_new

This creates a Pandas DataFrame out of the feature weights.

In [ ]:
item_new_2 = pandas.DataFrame(transposed_components)
item_new_2

This names the feautre weight colums on the fly.  We can change the number of topics without upsetting this.

In [ ]:
#This is a little script to name the colum names appropriately
col_number = n_topics
col_nubber_original = col_number
column_name_string_PS1_features = ""
for i in range(0,col_number):
    column_name_string_PS1_features+="'Topic_"+str(col_nubber_original-col_number)+"_PS1_features', "
    col_number = col_number - 1


#This cleans up the end of the string produced above and adds brackets
column_name_string_PS1_features = column_name_string_PS1_features[:-2]
column_name_string_PS1_features = '['+column_name_string_PS1_features+']'


#This converts the string into a list type object.
column_name_string_PS1_features = ast.literal_eval(column_name_string_PS1_features)

#This applies the string contining the names of the colums to the dataframe
item_new_2.columns = column_name_string_PS1_features

item_new_2

Now we can merge the two data frames we saw above: the features (terms), and the feature weights.

In [ ]:
df_crazy = pandas.concat([item_new, item_new_2], axis=1)
df_crazy

### Save the Feature Matrix

In [ ]:
#This saves the feature matrix to a csv
df_crazy.to_csv(path+'Admissions_PS1_'+topic_num_string+'_Topic_Feature_Words-'+time_for_f_name+'.csv', sep=',')

### Reviewing Final Model Output

Now we can sort the dataframe for the topic of interest, and view the top documents for the topics.
Below we sort the documents first by Topic 0 (looking at the top words for this topic I think it's about family, health, and domestic activities), and next by Topic 1 (again looking at the top words I think this topic is about children playing outside in nature). These topics may be a family/nature split?

Look at the titles for the two different topics. Look at the gender of the author. Hypotheses?

We can read individual essays in full using the code below.  Change the number in the final set of brackets to point to a spesific serial number (ID-1).

In [ ]:
print(df_w_topics[['CPID', 'PS1', 'Topic_0_PS1']].sort_values(by=['Topic_0_PS1'], ascending=False))

In [ ]:
print(df_w_topics['PS1'][205])

This allows us to sort the most imporant words/features for each topic.

In [ ]:
topic_0_feature_terms = df_crazy.sort_values(['Topic_2_PS1_features'], ascending=False)
topic_0_feature_terms

## 9. The End!

In [ ]:
# This calculates the amount of time it took to run the model, it requires "start_time" set at start.
end_time = time.time()
total_seconds = end_time - start_time

m, s = divmod(total_seconds, 60)
h, m = divmod(m, 60)
print("This script took %d:%02d:%02d to run (h:m:s)" % (h, m, s))

In [ ]:
print('Done! The output of the section above can be found in the folder '+time_for_f_name+'.csv')

## 10. Scratch Cell